In [6]:
import pandas as pd

In [7]:
chunk_iter = pd.read_sas("alltic_viobydy_q_052825_142505.sas7bdat",chunksize=10000, encoding = "utf-8")
alltic = next(chunk_iter)

In [12]:
for col in alltic.columns:
    print(col)

DATE
adjusted
volume
DGS10
DGS2
DGS3
DGS1
DGS3mo
DGS6mo
dgs2p5
dgs1p5
dgs15mo
dgs9mo
CPI
ppi
unemprate
laborpart
indprod
ffr
ycurve10y3m
ycurve10y2y
moodycorpbondyield
snp500rets
snp500
rollmean
rollmean1yr
rollstd
rollstd1yr
rollstd3mo
rollstd6mo
rollstd9mo
rollstd15mo
rollstd18mo
rollstd24mo
ticker
returns
mean_bb_price3mos1
mean_bb_price6mos1
mean_bb_price9mos1
mean_bb_price12mos1
mean_bb_price15mos1
mean_bb_price18mos1
mean_bb_price24mos1
mean_bb_price3mos2
mean_bb_price6mos2
mean_bb_price9mos2
mean_bb_price12mos2
mean_bb_price15mos2
mean_bb_price18mos2
mean_bb_price24mos2
mean_bb_price3mos3
mean_bb_price6mos3
mean_bb_price9mos3
mean_bb_price12mos3
mean_bb_price15mos3
mean_bb_price18mos3
mean_bb_price24mos3
mean_bb_price3mos4
mean_bb_price6mos4
mean_bb_price9mos4
mean_bb_price12mos4
mean_bb_price15mos4
mean_bb_price18mos4
mean_bb_price24mos4
mean_bbb_price24mos
mean_bbb_price30mos
mean_bbb_price36mos
pct_above3mos1
pct_above6mos1
pct_above9mos1
pct_above12mos1
pct_above15mos1
pct_a

In [158]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from typing import Dict, Any

# =============================================================================
# 0. GLOBAL CONFIGURATION & STATIC DATA (Moved to Global Scope)
# =============================================================================

# Configuration (equivalent to SAS macro variables)
NOTIONAL = 500000.0
PRICE_VOL_SCALING = 252  # Daily to Annual volatility scaling

# Barrier thresholds for the 4 scenarios
CTHRESHOLDS = {1: 0.55, 2: 0.65, 3: 0.75, 4: 0.85}
# Secondary thresholds for calculating 'ibelow' probabilities
MTHRESHOLDS = {1: 0.55, 2: 0.65, 3: 0.75, 4: 0.85}

# Static Data (equivalent to SAS arrays)
TIME_LABELS = {
    1: "3mos", 2: "6mos", 3: "9mos", 4: "12mos", 5: "15mos", 6: "18mos", 7: "24mos",
    8: "6mmos", 9: "9mmos", 10: "12mmos", 11: "15mmos", 12: "18mmos", 13: "24mmos"
}
TIME_PERIODS = {
    1: 0.25, 2: 0.50, 3: 0.75, 4: 1.00, 5: 1.25, 6: 1.50, 7: 2.00,
    8: 0.50, 9: 0.75, 10: 1.00, 11: 1.25, 12: 1.50, 13: 2.00
}
N_OBSERVATIONS = {
    1: 3, 2: 3, 3: 3, 4: 4, 5: 5, 6: 6, 7: 8,
    8: 6, 9: 9, 10: 12, 11: 15, 12: 18, 13: 24
}
# Risk-free rate mapping (SAS: rfrs_barr)
RFR_KEYS = {1: 'DGS3mo', 2: 'DGS6mo', 3: 'dgs9mo', 4: 'DGS1', 5: 'dgs15mo', 6: 'dgs1p5', 7: 'DGS2'}


# =============================================================================
# 1. HELPER FUNCTIONS (Translating SAS Logic)
# =============================================================================

def _calculate_bsm_payouts(
    fin_price: float,
    strike_price: float,
    time_to_maturity: float,
    risk_free_rate: float,
    volatility: float,
    digital_put_payoff: float
) -> float:
    """
    Calculates the structured note's coupon based on embedded options,
    replicating the Black-Scholes-Merton (BSM) logic from the SAS code.
    """
    if volatility <= 0 or fin_price <= 0 or time_to_maturity <= 0:
        return 0.0

    d1 = (np.log(fin_price / strike_price) + (risk_free_rate + volatility**2 / 2) * time_to_maturity) / (volatility * np.sqrt(time_to_maturity))
    d2 = d1 - volatility * np.sqrt(time_to_maturity)

    call_price = (fin_price * norm.cdf(d1) - strike_price * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(d2))
    put_price = call_price + strike_price * np.exp(-risk_free_rate * time_to_maturity) - fin_price
    digital_put_price = digital_put_payoff * np.exp(-risk_free_rate * time_to_maturity) * norm.cdf(-d2)

    coupon = np.exp(risk_free_rate * time_to_maturity) * (put_price + digital_put_price) / fin_price + (risk_free_rate * time_to_maturity)
    return coupon

def _calculate_aer(
    pr_a: float,
    pr_b: float,
    pr_ib: float,
    coupon: float,
    n_looks: int,
    notional: float,
    fin_price: float,
    mean_bb_price: float,
    num_ncalls: int = 0
) -> float:
    """
    Calculates the expected return based on probabilities of different outcomes.
    This is a direct translation of the core SAS calculation loop for a single scenario.
    """
    if coupon <= 0 or pd.isna(pr_a) or pd.isna(pr_b):
        return 0.0

    shares = notional / fin_price if fin_price > 0 else 0
    payout_constant1 = notional * coupon / n_looks
    payout_constant2 = (mean_bb_price - fin_price) * shares
    payout_constant3 = notional * coupon
    print(pr_a, pr_b)
    pr_g = 1.0 - pr_a - pr_b
    if pr_g < 0: pr_g = 0

    pr_b_last = pr_b - pr_ib

    n_looks_nc0 = n_looks - num_ncalls
    n_looks_nc1 = n_looks - 1 - num_ncalls
    n_looks_nc2 = n_looks - 2 - num_ncalls
    if n_looks_nc2 < 0: return 0.0

    sum_each_pay = 0.0

    if pr_a == 1.0:
        sum_each_pay = payout_constant1 * (num_ncalls + 1)
    elif pr_b == 1.0:
        sum_each_pay = payout_constant2
    else:
        prob = ((pr_b + pr_g) ** n_looks_nc1) * (pr_g + pr_a)
        sum_each_pay += prob * payout_constant3

        if pr_b != pr_b_last and pr_ib > 0:
            for i in range(n_looks_nc2 + 1):
                prob = ((pr_b + pr_g) ** i) * (pr_a + pr_g * (pr_b ** (n_looks_nc2 - i)) * pr_ib)
                payout = payout_constant1 * (i + 1) + payout_constant1 * num_ncalls
                sum_each_pay += prob * payout

            prob = (pr_b ** (n_looks - 1)) * pr_b_last
            sum_each_pay += prob * payout_constant2
            for i in range(1, num_ncalls + 1):
                prob = (pr_g + pr_a) * (pr_b ** (n_looks - 1 - i)) * pr_b_last
                payout = (payout_constant1 * i) + payout_constant2
                sum_each_pay += prob * payout
            for i in range(1, n_looks_nc1 + 1):
                prob = ((pr_b + pr_g) ** (i - 1)) * pr_g * (pr_b ** (n_looks_nc1 - i)) * pr_b_last
                payout = payout_constant1 * (num_ncalls + i) + payout_constant2
                sum_each_pay += prob * payout
            
            for i in range(1, num_ncalls + 1):
                 prob = (pr_g + pr_a) * (pr_b ** (n_looks - 1 - i)) * pr_ib
                 payout = payout_constant1 * i
                 sum_each_pay += prob * payout
        else:
            for i in range(n_looks_nc2 + 1):
                prob = ((pr_b + pr_g) ** i) * pr_a
                payout = payout_constant1 * (i + 1) + payout_constant1 * num_ncalls
                sum_each_pay += prob * payout

            prob = pr_b ** n_looks
            sum_each_pay += prob * payout_constant2
            for i in range(1, num_ncalls + 1):
                prob = (pr_g + pr_a) * (pr_b ** (n_looks - i))
                payout = (payout_constant1 * i) + payout_constant2
                sum_each_pay += prob * payout
            for i in range(1, n_looks_nc1 + 1):
                prob = ((pr_b + pr_g) ** (i - 1)) * pr_g * (pr_b ** (n_looks_nc0 - i))
                payout = payout_constant1 * (num_ncalls + i) + payout_constant2
                sum_each_pay += prob * payout

    return sum_each_pay / notional if notional > 0 else 0.0

# =============================================================================
# 2. MAIN PROCESSING FUNCTION
# =============================================================================

def calculate_all_note_scenarios(data_row: Dict[str, Any]) -> Dict[str, float]:
    """
    Main function to process a single row of data (one ticker at one date)
    and calculate the AER for all defined scenarios.
    """
    results = {}
    fin_price = data_row['adjusted']
    shares = NOTIONAL / fin_price if fin_price > 0 else 0

    for rr in range(1, 5):
        cthreshold = CTHRESHOLDS[rr]
        mthreshold = MTHRESHOLDS[rr]

        for ii in range(1, 14):
            time_label = TIME_LABELS[ii]
            time_period = TIME_PERIODS[ii]
            n_looks = N_OBSERVATIONS[ii]
            vol_and_rfr_key = min(ii, 7)
            
            vol_sas_label = {
                "3mos": "rollstd3mo", "6mos": "rollstd6mo", "9mos": "rollstd9mo",
                "12mos": "rollstd1yr", "15mos": "rollstd15mo", "18mos": "rollstd18mo",
                "24mos": "rollstd24mo"
            }
            # Fallback to 1yr vol if specific one not found
            vol_key = vol_sas_label.get(time_label.replace("mmos", "mos"), 'rollstd1yr')
            
            hist_std = data_row.get(vol_key, data_row['rollstd1yr']) * np.sqrt(PRICE_VOL_SCALING)
            rfr = data_row.get(RFR_KEYS[vol_and_rfr_key], 0) #/ 100.0

            strike_price = fin_price * cthreshold
            digital_payoff = fin_price * (1 - cthreshold)
            coupon = _calculate_bsm_payouts(fin_price, strike_price, time_period, rfr, hist_std, digital_payoff)
            
            tl_correct = time_label.replace("mmos", "mos")
            prob_above = data_row.get(f"pct_above{tl_correct}{rr}", 0)
            prob_below = data_row.get(f"pct_below{tl_correct}{rr}", 0)
            mean_bb_price = data_row.get(f"mean_bb_price{tl_correct}{rr}", 0)
            mean_bb_price = 0 if pd.isna(mean_bb_price) else mean_bb_price
            print(f"mean_bb_price{tl_correct}{rr}")
            # --- Calculate 'pct_ibelow' and 'pr_ib' ---
            # This logic assumes 'pct_ibelow' would be the probability of being below the 'mthreshold'
            # In a full model, this value would be pre-calculated. Here, we simulate it
            # based on the relationship between the two thresholds for a more realistic 'pr_ib'.
            pct_ibelow_simulated = prob_below * (mthreshold / cthreshold)**2 if cthreshold > 0 else 0
            pr_ib = max(prob_below - pct_ibelow_simulated, 0)
            
            # --- Strategic Non-Callable Logic (from SAS) ---
            num_ncalls = 0
            if ii == 2 or ii == 3: num_ncalls = 1 if rr == 1 else 0
            elif ii == 4: num_ncalls = 2 if rr == 1 else (1 if rr == 2 else 0)
            elif ii in [5, 6, 8]: num_ncalls = 2 if rr == 1 else (1 if rr >= 2 else 0)
            elif ii == 7 or ii == 9: num_ncalls = 3 if rr == 1 else (2 if rr >= 2 else 0)
            elif ii == 10: num_ncalls = 4 if rr == 1 else (3 if rr >= 2 else 0)
            elif ii == 11: num_ncalls = 5 if rr == 1 else (4 if rr == 2 else (3 if rr >= 3 else 0))
            elif ii == 12: num_ncalls = 6 if rr == 1 else (5 if rr >= 2 else 0)
            elif ii == 13: num_ncalls = 12 if rr == 1 else (8 if rr == 2 else (6 if rr >= 3 else 0))
            print(time_label)
            aer = _calculate_aer(
                prob_above, prob_below, pr_ib, coupon, n_looks,
                NOTIONAL, fin_price, mean_bb_price, num_ncalls
            )
            
            annualized_aer = aer / time_period if time_period > 0 else 0.0
            results[f"AER_{time_label}_b{rr}"] = annualized_aer
            
    return results


In [159]:
mean_bb_price3mos1

NameError: name 'mean_bb_price3mos1' is not defined

In [173]:
calculate_all_note_scenarios(alltic[0:].to_dict('records')[6975])

mean_bb_price3mos1
3mos
0.1111111111111111 0.0
mean_bb_price6mos1
6mos
0.05555555555555555 0.0
mean_bb_price9mos1
9mos
0.037037037037037035 0.0
mean_bb_price12mos1
12mos
0.027777777777777776 0.0
mean_bb_price15mos1
15mos
0.022222222222222223 0.0
mean_bb_price18mos1
18mos
0.019943019943019943 0.0
mean_bb_price24mos1
24mos
0.019943019943019943 0.0
mean_bb_price6mos1
6mmos
0.05555555555555555 0.0
mean_bb_price9mos1
9mmos
0.037037037037037035 0.0
mean_bb_price12mos1
12mmos
0.027777777777777776 0.0
mean_bb_price15mos1
15mmos
0.022222222222222223 0.0
mean_bb_price18mos1
18mmos
0.019943019943019943 0.0
mean_bb_price24mos1
24mmos
0.019943019943019943 0.0
mean_bb_price3mos2
3mos
0.1111111111111111 0.0
mean_bb_price6mos2
6mos
0.05555555555555555 0.0
mean_bb_price9mos2
9mos
0.037037037037037035 0.0
mean_bb_price12mos2
12mos
0.027777777777777776 0.0
mean_bb_price15mos2
15mos
0.022222222222222223 0.0
mean_bb_price18mos2
18mos
0.019943019943019943 0.0
mean_bb_price24mos2
24mos
0.019943019943019943 0

{'AER_3mos_b1': np.float64(0.048764678137442556),
 'AER_6mos_b1': np.float64(0.0517239831485344),
 'AER_9mos_b1': np.float64(0.050668656557772994),
 'AER_12mos_b1': np.float64(0.05117906227695616),
 'AER_15mos_b1': np.float64(0.06018468135954469),
 'AER_18mos_b1': np.float64(0.0630823725501202),
 'AER_24mos_b1': np.float64(0.06746586307430652),
 'AER_6mmos_b1': np.float64(0.04129409814296424),
 'AER_9mmos_b1': np.float64(0.04187856225860869),
 'AER_12mmos_b1': np.float64(0.04414215420552162),
 'AER_15mmos_b1': np.float64(0.054257168600415004),
 'AER_18mmos_b1': np.float64(0.058100406824476224),
 'AER_24mmos_b1': np.float64(0.0656042280941857),
 'AER_3mos_b2': np.float64(0.04959182976394631),
 'AER_6mos_b2': np.float64(0.05435202653208162),
 'AER_9mos_b2': np.float64(0.0568104659623517),
 'AER_12mos_b2': np.float64(0.06200444482493168),
 'AER_15mos_b2': np.float64(0.07783638372982186),
 'AER_18mos_b2': np.float64(0.0799418057906969),
 'AER_24mos_b2': np.float64(0.0818768058931221),
 'AE

In [174]:
alltic.iloc[6975].aer9mos3

np.float64(0.06740850587790694)

In [175]:
.02713897136329844/2

0.01356948568164922

In [157]:
alltic.iloc[0].mean_bb_price12mos4

np.float64(nan)